# Построение Дашборда для Яндекс.Дзен

In [1]:
import pandas as pd
import numpy as np
import time
import datetime
import os

#import IPython
import plotly

import plotly.express as px
import plotly.graph_objs as go
from plotly.subplots import make_subplots

#from scipy import stats as st
#import math as mth
#import re
import seaborn as sns
import warnings
#warnings.filterwarnings("ignore") 
pd.set_option('chained_assignment', None)

#import plotly.io as pio
#pio.renderers.default='notebook'

#import csv
#import requests
#from bs4 import BeautifulSoup

pd.set_option("max_colwidth", 200) # задаем максимальное количество отображаемых символов поля 
#pd.set_option("max_columns",10) # столбцов много, поэтому устанавливаем отображение всех, с запасом
pd.options.display.max_columns = 10

In [2]:
df = pd.read_csv('dash_visits.csv')

#df.drop('Unnamed: 0', axis=1, inplace=True)

df.head()

,record_id,item_topic,source_topic,age_segment,dt,visits
0,1040597,Деньги,Авто,18-25,2019-09-24 18:32:00,3
1,1040598,Деньги,Авто,18-25,2019-09-24 18:35:00,1
2,1040599,Деньги,Авто,18-25,2019-09-24 18:54:00,4
3,1040600,Деньги,Авто,18-25,2019-09-24 18:55:00,17
4,1040601,Деньги,Авто,18-25,2019-09-24 18:56:00,27


* `record_id` - уникальный идентификатор события
* `item_topic` - тема карточки
* `source_topic` - тема источника
* `age_segment` - возрастной сегмент
* `dt` - время события с точностью до минуты
* `visits` - количество посещений

In [3]:
# проверка дат событий
print(df['dt'].min(), '\t',df['dt'].max())

2019-09-24 18:28:00 	 2019-09-24 19:00:00


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30745 entries, 0 to 30744
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   record_id     30745 non-null  int64 
 1   item_topic    30745 non-null  object
 2   source_topic  30745 non-null  object
 3   age_segment   30745 non-null  object
 4   dt            30745 non-null  object
 5   visits        30745 non-null  int64 
dtypes: int64(2), object(4)
memory usage: 1.4+ MB


In [5]:
# проверка Stacked Area по темам карточек
fig = go.Figure()

for item in df['item_topic'].unique():
    current = df.query('item_topic == @item').groupby('dt')['visits'].sum()
    fig.add_trace(
        go.Scatter(
            x=current.index,
            y=current.values,
            mode='lines',
            stackgroup='one',
            name=item
                
        )
    )
    
fig.show()

In [6]:
# проверка процентов для Stacked Area по процентам
df['percent_of_visits'] = df['visits'] / sum(df['visits'])

round(df.groupby('item_topic')['percent_of_visits'].sum() * 100, 2).sort_values(ascending=False)

item_topic
Наука                 7.01
Отношения             6.66
Интересные факты      6.43
Общество              6.33
Подборки              5.73
Россия                5.47
Полезные советы       4.98
История               4.96
Семья                 3.84
Женщины               3.71
Дети                  3.54
Рассказы              3.52
Здоровье              3.35
Деньги                3.32
Культура              3.29
Красота               3.16
Туризм                3.07
Юмор                  3.03
Скандалы              3.00
Путешествия           2.99
Искусство             2.75
Психология            2.59
Женская психология    2.49
Шоу                   2.42
Знаменитости          2.38
Name: percent_of_visits, dtype: float64

In [7]:
# проверка процентов для Pie
round(df.groupby('source_topic')['percent_of_visits'].sum() * 100, 2).sort_values(ascending=False)

source_topic
Семейные отношения    10.74
Россия                 9.62
Полезные советы        8.84
Путешествия            7.78
Знаменитости           7.72
Кино                   6.47
Дети                   4.91
История                4.72
Семья                  4.48
Здоровье               4.14
Одежда                 3.83
Авто                   3.08
Искусство              2.58
Сад и дача             2.41
Политика               2.37
Еда                    2.22
Сделай сам             2.03
Психология             1.85
Ремонт                 1.84
Спорт                  1.69
Деньги                 1.66
Интерьеры              1.17
Технологии             1.13
Строительство          0.97
Музыка                 0.92
Финансы                0.85
Name: percent_of_visits, dtype: float64

In [8]:
# проверка корректности подсчета процентов для таблицы
df.query('item_topic == "Деньги" & source_topic == "Авто"')['visits'].sum() / sum(df.query('source_topic == "Авто"')['visits'])

0.035329779450193374

In [9]:
# вытаскиваем часы и минуты для дальнейшего отображения в качестве засечек на временной шкале в dash
pd.to_datetime(df['dt'].min(), format='%Y-%m-%d %H:%M:%S').strftime('%H:%M')

'18:28'

In [19]:
int(time.mktime(datetime.datetime.strptime(df['dt'].max(), '%Y-%m-%d %H:%M:%S').timetuple()))

1569340800

In [23]:
# преобразования timestamp для конвертации и расконвертации в разных форматах (ползунок времени в dash принимает значения только float, но не datetime)
time.mktime(datetime.datetime.strptime(df['dt'].max(), '%Y-%m-%d %H:%M:%S').timetuple())

datetime.datetime.fromtimestamp(1569340800).strftime('%Y-%m-%d %H:%M:%S')

datetime.datetime.fromtimestamp(1569340800).strftime('%H:%M')
#df.loc[1, 'dt']

'19:00'

In [28]:
df['dt'].min() + '15'

'2019-09-24 18:28:0015'

In [11]:
# преобразования для автоматического заполнения в .py скрипте для отрисовки засечек на временной шкале
min = 1569338880
max = 1569340800
delta = min

for i in range(10):
    delta += (max - min) / 10
    #print(int(delta), '\t', datetime.datetime.fromtimestamp(delta).strftime('%d.%m %H:%M'))
    print(str(int(delta))+': '+'{\'label\': '+'\'' + datetime.datetime.fromtimestamp(delta).strftime('%d.%m %H:%M')+'\'},')

1569339072: {'label': '24.09 18:31'},
1569339264: {'label': '24.09 18:34'},
1569339456: {'label': '24.09 18:37'},
1569339648: {'label': '24.09 18:40'},
1569339840: {'label': '24.09 18:44'},
1569340032: {'label': '24.09 18:47'},
1569340224: {'label': '24.09 18:50'},
1569340416: {'label': '24.09 18:53'},
1569340608: {'label': '24.09 18:56'},
1569340800: {'label': '24.09 19:00'},


In [12]:
# проценты для Stacked Area


total = df.groupby('dt') \
    .agg({'visits': 'sum'}) \
    .rename(columns = {'visits': 'total_visits'})

df_perc = df.set_index('dt') \
    .join(total) \
    .reset_index()

df_perc['visits'] = df_perc['visits'] / df_perc['total_visits']

# проверка Stacked Area по темам карточек
fig = go.Figure()

for item in df_perc['item_topic'].unique():
    current = df_perc.query('item_topic == @item').groupby('dt')['visits'].sum()
    fig.add_trace(
        go.Scatter(
            x=current.index,
            y=current.values,
            mode='lines',
            stackgroup='one',
            name=item
                
        )
    )
    
fig.show()

In [13]:
# круговая диаграмма для построения и проверки
fig = go.Figure()

df_source = df.groupby('source_topic')['visits'].sum()

fig.add_trace(
        go.Pie(
            labels = df_source.index,
            values=df_source.values
                
        )
    )
fig.show()

In [14]:
# таблица взаимосвязи источника и темы

df = pd.read_csv('dash_visits.csv')
pivot = df[['item_topic', 'source_topic', 'visits']].pivot_table(
    index='item_topic',
    columns='source_topic',
    values='visits',
    aggfunc='sum'
)
pivot = pivot.fillna(0,downcast='infer')

# сделаем аналог тепловой карты
cm = sns.light_palette("green", as_cmap=True)

pivot.style.background_gradient(cmap=cm)

source_topic,Авто,Деньги,Дети,Еда,Здоровье,Знаменитости,Интерьеры,Искусство,История,Кино,Музыка,Одежда,Полезные советы,Политика,Психология,Путешествия,Ремонт,Россия,Сад и дача,Сделай сам,Семейные отношения,Семья,Спорт,Строительство,Технологии,Финансы
item_topic,,,,,,,,,,,,,,,,,,,,,,,,,,
Деньги,338,203,859,193,65,591,62,172,271,1042,42,98,1087,949,305,536,362,1132,101,137,696,345,134,68,88,415
Дети,192,228,621,675,346,1444,97,238,455,474,9,320,1541,135,128,431,174,547,633,354,1184,558,21,103,28,53
Женская психология,26,256,613,67,349,546,0,96,181,264,5,466,418,23,463,321,13,660,49,19,2073,759,26,0,39,5
Женщины,156,122,875,348,1022,967,175,485,560,340,45,547,986,78,253,192,67,397,125,223,2270,988,210,56,6,6
Здоровье,232,95,798,558,404,962,57,64,68,156,20,518,2335,121,214,393,145,712,542,548,814,247,197,106,74,19
Знаменитости,29,55,145,36,139,445,12,356,644,408,55,467,178,35,88,299,3,2275,21,6,764,234,433,50,158,59
Интересные факты,1254,145,327,391,2090,720,54,697,1273,1033,325,837,2023,340,449,1448,175,2567,825,341,1323,611,186,234,219,55
Искусство,145,98,639,7,70,1083,164,512,596,1254,223,209,331,121,98,664,172,632,43,128,754,404,54,53,58,4
История,522,114,1047,60,771,921,30,695,636,1091,164,310,859,695,227,2643,168,1491,209,149,1344,696,190,9,286,62


In [15]:
# тепловая карта взаимосвязи источника и темы карточек
fig = go.Figure(
        data=go.Heatmap(
            x=df['source_topic'],
            y=df['item_topic'],
            z=df['visits'],
            colorscale=[
                [0, 'rgb(255, 255, 204)'],
                [0.13, 'rgb(255, 237, 160)'],
                [0.25, 'rgb(254, 217, 118)'],
                [0.38, 'rgb(254, 178, 76)'],
                [0.5, 'rgb(253, 141, 60)'],
                [0.63, 'rgb(252, 78, 42)'],
                [0.75, 'rgb(227, 26, 28)'],
                [0.88, 'rgb(189, 0, 38)'],
                [1.0, 'rgb(128, 0, 38)']
            ],
            zauto=True,
            hovertemplate='Карточка: %{y}<br>Источник: %{x}<br>Посещений: %{z}',
            name = '',
        )
    )

fig.show()